In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

from sentence_transformers import SentenceTransformer, util

c:\Users\citak\anaconda3\envs\pytorch\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
csv_path = "test_questions.csv"
df_original = pd.read_csv(csv_path)

questions_orig = df_original["Questions"].tolist()

df_original.head()

,Questions
0,What is the maximum deposit amount a Recurrin...
1,How can I prematurely liquidate my Fixed Depo...
2,What is IVR Password and how does it work
3,What are the available plans under the Critic...
4,How much is the Service Tax on Cross Currency...


In [3]:
len(questions_orig)

249

In [4]:
variations_1_path = "test_questions_variations.txt"

df_variations_1_oriq = []
df_variations_1_var = []

with open(variations_1_path, "r") as file:
    df_variations_1 = file.readlines()

    for line in df_variations_1:
        orig_q = line.split("→")[0].replace("\"",'').strip()
        var_q = line.split("→")[1].replace("\"",'').strip()

        df_variations_1_oriq.append(orig_q)
        df_variations_1_var.append(var_q)

In [5]:
len(df_variations_1)

249

In [6]:
len(df_variations_1_oriq), len(df_variations_1_var)

(249, 249)

### Now to be ensure, try to compare each original and variation question

In [7]:
# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
similarity_matrix = np.ones(len(questions_orig))*-1

In [9]:
len(questions_orig), len(df_variations_1_var)

(249, 249)

In [10]:
for idx, (orig_q, var_q) in tqdm(enumerate(zip(questions_orig, df_variations_1_var))):
    original_question = orig_q.replace("\'", '').strip()
    
    # compare the questions
    embedding1 = model.encode(original_question, convert_to_tensor=True)
    embedding2 = model.encode(var_q, convert_to_tensor=True)

    # Compute cosine similarity
    similarity = util.pytorch_cos_sim(embedding1, embedding2)
    similarity = similarity.detach().cpu().numpy()[0,0]

    similarity_matrix[idx] = similarity

0it [00:00, ?it/s]c:\Users\citak\anaconda3\envs\pytorch\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
249it [00:04, 50.84it/s]


In [11]:
for idx in range(len(similarity_matrix)):
    if similarity_matrix[idx] < 0.5:
        print(f"IDX: {idx}")
        try:
            print(questions_orig[idx])
            print(df_variations_1_var[idx])
            print(similarity_matrix[idx])
            print("-"*20)
        except:
            print("Exception")
            print("-"*20)

IDX: 27
 What are the advantages of investing in SIP
What benefits do I gain by choosing a Systematic Investment Plan over lump-sum investments?
0.43533602356910706
--------------------
IDX: 69
 When do I become liable for TDS
Under what circumstances does my account become subject to Tax Deduction at Source?
0.24544572830200195
--------------------
IDX: 78
 Cab sum assured be extended
Is it possible to increase the sum assured amount after policy issuance?
0.47079604864120483
--------------------
IDX: 89
 What is the policy term of the plan
For what duration does this insurance plan provide coverage?
0.4790274500846863
--------------------
IDX: 96
 What will I be getting on maturity
What benefits or amount will I receive when my policy reaches its maturity date?
0.4833115339279175
--------------------
IDX: 190
 Is there any Surrender benefit under this product
Does this insurance plan offer any value if I decide to terminate it early?
0.23887720704078674
--------------------
IDX: 195
